In [1]:
import csv
import numpy as np

In [2]:
with open('input.csv', 'r') as csvfile:
    inputs = list(csv.reader(csvfile))
    inputs = [
        [float(num) for num in row]
        for row in inputs
        if '-999' not in row
    ]
inputs = np.array(inputs)

with open('output.csv', 'r') as csvfile:
    Y = list(csv.reader(csvfile))[1:] # Skip the first row
    Y = [
        [float(num) for num in row]
        for row in Y
    ]
Y = np.array(Y)

Get the depths with output values

In [3]:
# Output depths
out_depths = Y[:,0]
out_depths = [round(depth) for depth in out_depths]

# Input depths
in_depths = inputs[:,0]
in_depths = [round(depth) for depth in in_depths]

Y = Y[:,-1]
print(Y)

[1.53757734e+01 6.40763963e-01 2.90368342e+00 2.42913986e+00
 9.66461146e+01 3.64993559e+01 1.01580394e+03 1.42019440e+03
 3.04064301e+00 2.40163804e+00 1.77057279e+00 2.04758087e+00
 1.79422727e+00 1.05591821e+00 2.48570121e+00 1.60532316e+01
 3.53483894e+01]


In [4]:
X = []
inputs = inputs[:,1:]
# Map output values to inputs
for depth in out_depths:
    if depth in in_depths:
        # Map the inputs to outputs by depth; remove the first redundant column
        X.append(inputs[in_depths.index(depth)])
X = np.array(X)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(inputs)
X = scaler.transform(X)
print(X)

[[-1.37691677  0.38663725 -0.59729995  0.19792547 -1.44899854]
 [-1.23954899  0.0137692  -0.92372995 -0.30620164 -1.01570437]
 [-1.10218122 -0.51426327 -0.92333393 -0.12101209  0.74106755]
 [-1.05341497 -0.12860334  1.45097117  0.02688235  1.54866783]
 [-0.7924162  -0.21233996 -1.02608271 -0.16988155 -1.3299499 ]
 [-0.7924162  -0.21233996 -1.02608271 -0.16988155 -1.3299499 ]
 [-0.7924162  -0.21233996 -1.02608271 -0.16988155 -1.3299499 ]
 [-0.7924162  -0.21233996 -1.02608271 -0.16988155 -1.3299499 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.78623533 -0.13521184 -0.05746864 -0.02841731 -1.0189219 ]
 [-0.77867942 -0.2122047  -0.95811155 -0.49139119 -0.99

In [6]:
from sklearn.neural_network import MLPRegressor

models = []
for hidden_size in range(1, 30):
    hidden_size = hidden_size * 10
    
    for lr in [0.1, 0.01, 0.001]:
        for alpha in [0.05, 0.1, 0.25, 0.5]:
            model = MLPRegressor(hidden_layer_sizes=(200,), 
                                 activation='relu', 
                                 solver='adam',
                                 alpha=alpha,
                                 max_iter=500,
                                 learning_rate_init=0.01,
                                 tol=0,
                                 validation_fraction=0,
                                 verbose=False)
            model.fit(X, Y)
            score = model.score(X,Y)
            print('Testing with hidden_size=%s, lr=%s, alpha:%s, score:%s' % (hidden_size, lr, alpha, score))
            models.append({'model': model, 'score': score})

/Users/2359media/anaconda/envs/yolo-keras/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Testing with hidden_size=10, lr=0.1, alpha:0.05, score:0.46256464519755547
Testing with hidden_size=10, lr=0.1, alpha:0.1, score:0.4645237705766678
Testing with hidden_size=10, lr=0.1, alpha:0.25, score:0.46295170749104325
Testing with hidden_size=10, lr=0.1, alpha:0.5, score:0.4629424962880422
Testing with hidden_size=10, lr=0.01, alpha:0.05, score:0.46065044923244436
Testing with hidden_size=10, lr=0.01, alpha:0.1, score:0.460768366318047
Testing with hidden_size=10, lr=0.01, alpha:0.25, score:0.4602170303827304
Testing with hidden_size=10, lr=0.01, alpha:0.5, score:0.46291230868748345
Testing with hidden_size=10, lr=0.001, alpha:0.05, score:0.46192082121166117
Testing with hidden_size=10, lr=0.001, alpha:0.1, score:0.46481826807240545
Testing with hidden_size=10, lr=0.001, alpha:0.25, score:0.4615727069099109
Testing with hidden_size=10, lr=0.001, alpha:0.5, score:0.46220033062054405
Testing with hidden_size=20, lr=0.1, alpha:0.05, score:0.462415949937915
Testing with hidden_size=20

Testing with hidden_size=100, lr=0.1, alpha:0.25, score:0.46250486159496873
Testing with hidden_size=100, lr=0.1, alpha:0.5, score:0.46373059900770697
Testing with hidden_size=100, lr=0.01, alpha:0.05, score:0.46456998908776775
Testing with hidden_size=100, lr=0.01, alpha:0.1, score:0.4630569897839365
Testing with hidden_size=100, lr=0.01, alpha:0.25, score:0.4618180692328674
Testing with hidden_size=100, lr=0.01, alpha:0.5, score:0.4622805727675617
Testing with hidden_size=100, lr=0.001, alpha:0.05, score:0.4601288333888219
Testing with hidden_size=100, lr=0.001, alpha:0.1, score:0.4638061616919326
Testing with hidden_size=100, lr=0.001, alpha:0.25, score:0.464290558775427
Testing with hidden_size=100, lr=0.001, alpha:0.5, score:0.4632844063004748
Testing with hidden_size=110, lr=0.1, alpha:0.05, score:0.4609523628122986
Testing with hidden_size=110, lr=0.1, alpha:0.1, score:0.46208990743650097
Testing with hidden_size=110, lr=0.1, alpha:0.25, score:0.4603017227739106
Testing with hid

Testing with hidden_size=190, lr=0.1, alpha:0.25, score:0.4626449828394596
Testing with hidden_size=190, lr=0.1, alpha:0.5, score:0.46382997025690487
Testing with hidden_size=190, lr=0.01, alpha:0.05, score:0.4620725950730279
Testing with hidden_size=190, lr=0.01, alpha:0.1, score:0.46335666324922575
Testing with hidden_size=190, lr=0.01, alpha:0.25, score:0.4626032532783038
Testing with hidden_size=190, lr=0.01, alpha:0.5, score:0.46164939145173445
Testing with hidden_size=190, lr=0.001, alpha:0.05, score:0.46264163555569376
Testing with hidden_size=190, lr=0.001, alpha:0.1, score:0.46305492534699966
Testing with hidden_size=190, lr=0.001, alpha:0.25, score:0.45994377737821623
Testing with hidden_size=190, lr=0.001, alpha:0.5, score:0.46203988900813414
Testing with hidden_size=200, lr=0.1, alpha:0.05, score:0.46173816165380044
Testing with hidden_size=200, lr=0.1, alpha:0.1, score:0.4616915571095159
Testing with hidden_size=200, lr=0.1, alpha:0.25, score:0.4623365006802843
Testing wit

Testing with hidden_size=280, lr=0.1, alpha:0.25, score:0.4622254699484218
Testing with hidden_size=280, lr=0.1, alpha:0.5, score:0.4634759848683346
Testing with hidden_size=280, lr=0.01, alpha:0.05, score:0.46189961865258744
Testing with hidden_size=280, lr=0.01, alpha:0.1, score:0.4643597320732195
Testing with hidden_size=280, lr=0.01, alpha:0.25, score:0.4625021987615731
Testing with hidden_size=280, lr=0.01, alpha:0.5, score:0.46201034846236866
Testing with hidden_size=280, lr=0.001, alpha:0.05, score:0.46193083608952984
Testing with hidden_size=280, lr=0.001, alpha:0.1, score:0.4615976306585464
Testing with hidden_size=280, lr=0.001, alpha:0.25, score:0.463498034040473
Testing with hidden_size=280, lr=0.001, alpha:0.5, score:0.4632705139976646
Testing with hidden_size=290, lr=0.1, alpha:0.05, score:0.4629167186552683
Testing with hidden_size=290, lr=0.1, alpha:0.1, score:0.4610734854476725
Testing with hidden_size=290, lr=0.1, alpha:0.25, score:0.4638736920745019
Testing with hidd

In [11]:
# Best score
models = sorted(models, key=lambda x:-x['score'])
top_model = models[0]['model']
print(top_model)

MLPRegressor(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200,), learning_rate='constant',
       learning_rate_init=0.01, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0, validation_fraction=0,
       verbose=False, warm_start=False)


In [8]:
print(Y)

[1.53757734e+01 6.40763963e-01 2.90368342e+00 2.42913986e+00
 9.66461146e+01 3.64993559e+01 1.01580394e+03 1.42019440e+03
 3.04064301e+00 2.40163804e+00 1.77057279e+00 2.04758087e+00
 1.79422727e+00 1.05591821e+00 2.48570121e+00 1.60532316e+01
 3.53483894e+01]


In [9]:
print(top_model.predict(X))

[ 30.59219221 -23.68771592   3.17534493   2.33203183 631.41888866
 631.41888866 631.41888866 631.41888866   3.33918531   3.33918531
   3.33918531   3.33918531   3.33918531   3.33918531   3.33918531
  50.68960678  50.68960678]


In [10]:
top_model.score(X, Y)

0.46635797327001105